## Analysis Workflow

The analysis proceeds through six systematic steps, each handling a specific aspect of performance evaluation:

1. **Loading Results**: Extract metrics from `results.json` or model-specific files
2. **DuckDB Query**: Fast SQL-based retrieval of fold-wise metrics
3. **MLflow Integration**: Load experiment tracking data (if available)
4. **Training Curves**: Extract epoch-wise or iteration-wise metrics from all folds
5. **Visualizations**: Generate comprehensive plots and charts
6. **Performance Summary**: Display detailed metrics breakdown

Each step is independent, allowing partial analysis even if some data sources are unavailable.


## Imports and Configuration

### Standard Library Imports

Essential Python standard library modules for file I/O, path manipulation, type hints, and system operations.

In [1]:
import json
import sys
from pathlib import Path
from typing import Dict, List, Optional, Any
import warnings

### Project Root Path Setup

Add project root and notebooks directory to Python path for importing project-specific utilities.

In [2]:
# Resolve project root by searching for lib/ directory
project_root = Path.cwd()
for _ in range(10):
    if (project_root / 'lib').exists() and (project_root / 'lib' / '__init__.py').exists():
        break
    parent = project_root.parent
    if parent == project_root:
        break
    project_root = parent

# Add paths to sys.path
sys.path.insert(0, str(project_root))
sys.path.insert(0, str(project_root / "src" / "notebooks"))

# Verify notebook_utils can be imported
try:
    import notebook_utils
    print(f"✓ Project root: {project_root}")
    print(f"✓ notebook_utils found at: {notebook_utils.__file__}")
except ImportError as e:
    print(f"⚠ Warning: {e}")
    print(f"Project root: {project_root}")
    print(f"Looking for: {project_root / 'src' / 'notebooks' / 'notebook_utils.py'}")


✓ Project root: /Users/santoshdesai/Downloads/fvc
✓ notebook_utils found at: /Users/santoshdesai/Downloads/fvc/src/notebooks/notebook_utils.py


### Third-Party Library Imports

Data science and visualization libraries for numerical computations, data manipulation, and statistical visualization.

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

### Plotting Configuration

Configure matplotlib and seaborn for publication-quality plots with appropriate DPI and styling.

In [4]:
warnings.filterwarnings('ignore')
sns.set_style("whitegrid")
plt.rcParams['figure.dpi'] = 100
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['font.size'] = 10

### Project-Specific Utilities Import

Import utility functions from `notebook_utils` module for data loading, metric extraction, and visualization.

In [5]:
try:    from notebook_utils import (        get_model_data_path, load_results_json, load_mlflow_metrics_by_model_type,        query_duckdb_metrics, get_metrics_data, display_duckdb_metrics_summary, plot_training_curves, find_roc_pr_curve_files,        display_roc_pr_curve_images, plot_validation_metrics_across_folds,        MODEL_TYPE_MAPPING    )except ImportError as e:    print(f"[ERROR] Failed to import notebook_utils: {e}")    sys.exit(1)

SyntaxError: invalid syntax (1957820862.py, line 1)

### Project Root Detection Function

Locate the project root directory by searching for the `lib/` folder marker.

In [6]:
def get_project_root() -> Path:
    """Find project root directory."""
    current = Path.cwd()
    for _ in range(10):
        if (current / "lib").exists() and (current / "lib" / "__init__.py").exists():
            return current
        parent = current.parent
        if parent == current:
            break
        current = parent
    return Path.cwd()


### Initialize Project Root

Call the project root detection function and display the resolved path.

In [7]:
PROJECT_ROOT = get_project_root()
print(f"Project root: {PROJECT_ROOT}")

Project root: /Users/santoshdesai/Downloads/fvc


## Analysis Workflow

The analysis proceeds through six systematic steps, each handling a specific aspect of performance evaluation:

1. **Loading Results**: Extract metrics from `results.json` or model-specific files
2. **DuckDB Query**: Fast SQL-based retrieval of fold-wise metrics
3. **MLflow Integration**: Load experiment tracking data (if available)
4. **Training Curves**: Extract epoch-wise or iteration-wise metrics from all folds
5. **Visualizations**: Generate comprehensive plots and charts
6. **Performance Summary**: Display detailed metrics breakdown

Each step is independent, allowing partial analysis even if some data sources are unavailable.


### Step 1/6: Loading Results from results.json

Load results from `results.json` which contains best hyperparameters found during grid search, best validation F1 score, and cross-validation results with mean and standard deviation metrics across folds.

In [8]:
# Initialize analysis variables
project_root = get_project_root()
model_id = "5alpha"
model_name = "Sklearn Logistic Regression"
print("=" * 70)
print("Sklearn Logistic Regression (5alpha) - Comprehensive Performance Analysis")
print("=" * 70)
print()

Sklearn Logistic Regression (5alpha) - Comprehensive Performance Analysis



In [9]:
print("[1/6] Loading results.json...")
model_path = get_model_data_path(model_id, project_root, MODEL_TYPE_MAPPING)
if not model_path:
    print(f"[ERROR] Model path not found for {model_id}")
    
results = load_results_json(model_id, project_root, MODEL_TYPE_MAPPING)
if results:
    print(f"  ✓ Loaded results.json")
    if "best_val_f1" in results:
        print(f"    - Best Val F1: {results['best_val_f1']:.4f}")
    if "cv_val_f1" in results:
        print(f"    - CV Val F1: {results['cv_val_f1']:.4f} ± {results.get('cv_val_f1_std', 0):.4f}")
else:
    print(f"  [WARN] No results.json found")
    results = {}

[1/6] Loading results.json...


NameError: name 'get_model_data_path' is not defined

### Step 2/6: Querying DuckDB for Fold-wise Metrics

Query the DuckDB analytics database for fast SQL-based retrieval of fold-wise metrics and aggregated statistics.

In [10]:
print("\n[2/6] Querying DuckDB for fold-wise metrics...")metrics_data = get_metrics_data(model_id, project_root)if metrics_data:    display_duckdb_metrics_summary(metrics_data, model_id)    # Convert to dict for backward compatibility with rest of notebook    duckdb_metrics = metrics_data.to_dict()else:    duckdb_metrics = {}

SyntaxError: invalid syntax (1899669095.py, line 1)

### Step 3/6: Loading MLflow Tracking Data

Attempt to load MLflow experiment tracking data. Note: Baseline sklearn models typically don't use MLflow tracking, so this step will return an informative message.

In [11]:
print("\n[3/6] Loading MLflow tracking data...")
model_type = MODEL_TYPE_MAPPING.get(model_id)
if model_type:
    mlflow_data = load_mlflow_metrics_by_model_type(model_type, "mlruns/", project_root)
    if mlflow_data and isinstance(mlflow_data, dict) and "message" not in mlflow_data:
        print(f"  ✓ Loaded MLflow data")
    else:
        print(f"  [INFO] Baseline models don't use MLflow tracking")
else:
    mlflow_data = {}


[3/6] Loading MLflow tracking data...


NameError: name 'MODEL_TYPE_MAPPING' is not defined

### Step 4/6: Extracting Training Curves

Check for ROC/PR curves at the root level. Sklearn models may not have fold-specific metrics.jsonl files, but typically have visualization plots at the model root directory.

In [12]:
print("\n[4/6] Extracting training curves...")
roc_pr_file = model_path / "roc_pr_curves.png"
if roc_pr_file.exists():
    print(f"  ✓ Found ROC/PR curves at root level")


[4/6] Extracting training curves...


NameError: name 'model_path' is not defined

### Step 5/6: Generating Comprehensive Visualizations

Generate visualizations including ROC/PR curves and validation metrics across folds if available.

In [13]:
print("\n[5/6] Generating comprehensive visualizations...")curve_files = find_roc_pr_curve_files(model_id, project_root, MODEL_TYPE_MAPPING)if curve_files:    print(f"  ✓ Found ROC/PR curve files")    display_roc_pr_curve_images(curve_files, model_name)    if duckdb_metrics and "fold_results" in duckdb_metrics:    print(f"  ✓ Plotting validation metrics across folds...")    fig = plot_validation_metrics_across_folds(duckdb_metrics, model_name)    if fig:        plt.show()  # Display the plot in notebook

SyntaxError: invalid syntax (3518120784.py, line 1)

### Step 6/6: Performance Summary

Display comprehensive performance summary including best hyperparameters, cross-validation results, and aggregated metrics.

In [14]:
print("\n[6/6] Performance Summary")
print("=" * 70)
    
if results:
    print("\nCross-Validation Results:")
    if "best_val_f1" in results:
        print(f"  Best Val F1:        {results['best_val_f1']:.4f}")
    if "cv_val_f1" in results:
        print(f"  CV Val F1:          {results['cv_val_f1']:.4f} ± {results.get('cv_val_f1_std', 0):.4f}")
    if "cv_val_auc" in results:
        print(f"  CV Val AUC:         {results['cv_val_auc']:.4f} ± {results.get('cv_val_auc_std', 0):.4f}")
    if "best_params" in results:
        print(f"  Best Parameters:    {results['best_params']}")
    
print("\n" + "=" * 70)
print("Analysis complete!")
print("=" * 70)


[6/6] Performance Summary


NameError: name 'results' is not defined

## Execute Analysis

Execute the cells below sequentially to perform the complete analysis. The analysis will:

1. Load results and metrics from multiple data sources
2. Query DuckDB for comprehensive fold-wise statistics
3. Attempt to load MLflow tracking data (if available)
4. Extract and visualize training curves from all folds
5. Generate comprehensive visualizations (ROC/PR curves, validation metrics, analysis plots)
6. Display detailed performance summary with aggregated statistics

**Note**: Execution may take a few moments as it loads data from multiple sources, generates visualizations, and performs statistical aggregations. All plots will be displayed inline in the notebook.
